In [1]:
import cv2
import numpy as np
import face_recognition
from keras.models import load_model

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load your trained models
model_vgg16 = load_model("C:\\Projects\\Python\\ELC\\VGG16_New.keras")
model_vgg19 = load_model("C:\\Projects\\Python\\ELC\\VGG19_New.keras")
model_DenseNet = load_model("C:\\Projects\\Python\\ELC\\DenseNet121_New.keras")

# Define labels for facial expression detection
labels = ['closed eyes', 'drowsy', 'half-closed', 'neutral', 'yawning']

# Function to preprocess frames for VGG models
def preprocess_frame(frame, target_size=(224, 224)):
    frame = cv2.resize(frame, target_size)
    frame = frame.astype('float32') / 255.0
    frame = np.expand_dims(frame, axis=0)
    return frame

# Function to perform ensemble prediction
def ensemble_predictions(preds_vgg16, preds_vgg19, preds_DenseNet):
    preds = np.array([preds_vgg16, preds_vgg19, preds_DenseNet])
    summed_preds = np.sum(preds, axis=0)
    averaged_preds = summed_preds / 3
    return averaged_preds, np.argmax(averaged_preds)

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate Mouth Aspect Ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[2] - mouth[10])  # upper_lip_top - lower_lip_bottom
    B = np.linalg.norm(mouth[4] - mouth[8])   # upper_lip_bottom - lower_lip_top
    C = np.linalg.norm(mouth[0] - mouth[6])   # left_corner - right_corner
    mar = (A + B) / (2.0 * C)
    return mar

# Function to calculate head tilt using nose landmarks
def calculate_head_tilt(nose_tip, chin):
    delta_x = chin[0] - nose_tip[0]
    delta_y = chin[1] - nose_tip[1]
    angle = np.arctan2(delta_y, delta_x) * 180 / np.pi
    return angle

# Capture video from webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Frame rate (frames per second)
frame_rate = 30

# Calculate the interval between frames in milliseconds
frame_interval = int(1000 / frame_rate)

# Threshold for mouth aspect ratio to determine if mouth is open
mouth_open_threshold = 0.20  # Adjust as needed

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture image")
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces using Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Detect landmarks using face_recognition
        face_landmarks_list = face_recognition.face_landmarks(face)
        if not face_landmarks_list:
            continue
        
        face_landmarks = face_landmarks_list[0]
        
        # Get the landmarks for the eyes, mouth, nose, and chin
        left_eye = np.array(face_landmarks['left_eye'])
        right_eye = np.array(face_landmarks['right_eye'])
        top_lip = np.array(face_landmarks['top_lip'])
        bottom_lip = np.array(face_landmarks['bottom_lip'])
        nose_tip = np.array(face_landmarks['nose_tip'][2])
        chin = np.array(face_landmarks['chin'][8])
        mouth = np.concatenate((top_lip, bottom_lip), axis=0)
        
        # Calculate EAR
        ear_left = eye_aspect_ratio(left_eye)
        ear_right = eye_aspect_ratio(right_eye)
        
        # Calculate MAR
        mar = mouth_aspect_ratio(mouth)
        
        # Calculate head tilt
        head_tilt_angle = calculate_head_tilt(nose_tip, chin)
        
 # Determine the label based on EAR, MAR, and head tilt
        if mar > mouth_open_threshold:
            label = 'yawning'
        elif ear_left > 0.23 and ear_right > 0.23:
            label = 'neutral'
        # elif ear_left < 0.20 and ear_right < 0.20 :
        #     label = 'CLOSED_EYES'
        elif ear_left < 0.20 and ear_right < 0.20 and head_tilt_angle > 15:  # Adjust the threshold for head tilt as needed
            label = 'drowsy'            
        elif 0.20 <= ear_left < 0.23 or 0.2 <= ear_right < 0.23:
            label = 'half-closed'
        else:
            face_preprocessed = preprocess_image(face)
            
            # Predict the label using ensembled models
            preds_vgg16 = model_vgg16.predict(face_preprocessed)
            preds_vgg19 = model_vgg19.predict(face_preprocessed)
            preds_DenseNet = model_DenseNet.predict(face_preprocessed)
            
            averaged_preds, final_prediction = ensemble_predictions(preds_vgg16, preds_vgg19, preds_DenseNet)
            
            # Check if final_prediction index is within valid range
            if final_prediction < len(labels):
                label = labels[final_prediction]
            else:
                label = "Unknown"
        
        # Display the label on the frame
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(frame_interval) & 0xFF == ord('q'):
        break

# Release the capture and destroy all windows

cap.release()
cv2.destroyAllWindows()


NameError: name 'preprocess_image' is not defined

In [1]:
import cv2
import numpy as np
import face_recognition
from keras.models import load_model

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load your trained models
model_vgg16 = load_model("C:\\Projects\\Python\\ELC\\VGG16_New.keras")
model_vgg19 = load_model("C:\\Projects\\Python\\ELC\\VGG19_New.keras")
model_DenseNet = load_model("C:\\Projects\\Python\\ELC\\DenseNet121_New.keras")

# Define labels for facial expression detection
labels = ['closed eyes', 'drowsy', 'half-closed', 'neutral', 'yawning']

# Function to preprocess frames for VGG models
def preprocess_frame(frame, target_size=(224, 224)):
    frame = cv2.resize(frame, target_size)
    frame = frame.astype('float32') / 255.0
    frame = np.expand_dims(frame, axis=0)
    return frame

# Function to perform ensemble prediction
def ensemble_predictions(preds_vgg16, preds_vgg19, preds_DenseNet):
    preds = np.array([preds_vgg16, preds_vgg19, preds_DenseNet])
    summed_preds = np.sum(preds, axis=0)
    averaged_preds = summed_preds / 3
    return averaged_preds, np.argmax(averaged_preds)

# Function to calculate Eye Aspect Ratio (EAR)
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Function to calculate Mouth Aspect Ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[2] - mouth[10])  # upper_lip_top - lower_lip_bottom
    B = np.linalg.norm(mouth[4] - mouth[8])   # upper_lip_bottom - lower_lip_top
    C = np.linalg.norm(mouth[0] - mouth[6])   # left_corner - right_corner
    mar = (A + B) / (2.0 * C)
    return mar

# Function to calculate head tilt using nose landmarks
def calculate_head_tilt(nose_tip, chin):
    delta_x = chin[0] - nose_tip[0]
    delta_y = chin[1] - nose_tip[1]
    angle = np.arctan2(delta_y, delta_x) * 180 / np.pi
    return angle

# Capture video from webcam
cap = cv2.VideoCapture(0)  # Use 0 for the default webcam

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Frame rate (frames per second)
frame_rate = 30

# Calculate the interval between frames in milliseconds
frame_interval = int(1000 / frame_rate)
# Threshold for mouth aspect ratio to determine if mouth is open
mouth_open_threshold = 0.17  # Adjust as needed

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture image")
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces using Haar Cascade
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255,255, 255), 2)
        
        # Detect landmarks using face_recognition
        face_landmarks_list = face_recognition.face_landmarks(face)
        if not face_landmarks_list:
            continue
        
        face_landmarks = face_landmarks_list[0]
        
        # Get the landmarks for the eyes, mouth, nose, and chin
        left_eye = np.array(face_landmarks['left_eye'])
        right_eye = np.array(face_landmarks['right_eye'])
        top_lip = np.array(face_landmarks['top_lip'])
        bottom_lip = np.array(face_landmarks['bottom_lip'])
        nose_tip = np.array(face_landmarks['nose_tip'][2])
        chin = np.array(face_landmarks['chin'][8])
        mouth = np.concatenate((top_lip, bottom_lip), axis=0)
        
        # Calculate EAR
        ear_left = eye_aspect_ratio(left_eye)
        ear_right = eye_aspect_ratio(right_eye)
        
        # Calculate MAR
        mar = mouth_aspect_ratio(mouth)
        
        # Calculate head tilt
        head_tilt_angle = calculate_head_tilt(nose_tip, chin)
        
        # Determine the label based on EAR, MAR, and head tilt
        if mar > mouth_open_threshold:
            label = 'yawning'
        elif ear_left > 0.23 and ear_right > 0.23:
            label = 'neutral'
        elif ear_left < 0.20 and ear_right < 0.20 and head_tilt_angle > 15:  # Adjust the threshold for head tilt as needed
            label = 'drowsy'            
        elif 0.20 <= ear_left < 0.23 or 0.2 <= ear_right < 0.23:
            label = 'half-closed'
        else:
            face_preprocessed = preprocess_frame(face)
            
            # Predict the label using ensembled models
            preds_vgg16 = model_vgg16.predict(face_preprocessed)
            preds_vgg19 = model_vgg19.predict(face_preprocessed)
            preds_DenseNet = model_DenseNet.predict(face_preprocessed)
            
            averaged_preds, final_prediction = ensemble_predictions(preds_vgg16, preds_vgg19, preds_DenseNet)
            
            # Check if final_prediction index is within valid range
            if final_prediction < len(labels):
                label = labels[final_prediction]
            else:
                label = "Unknown"
        
        # Display the label on the frame
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        
        # Mark points on eyes and mouth
        for (ex, ey) in left_eye:
            cv2.circle(face, (ex, ey), 2, (144,238,144), -1)
        for (ex, ey) in right_eye:
            cv2.circle(face, (ex, ey), 2, (144,238,144), -1)
        for (ex, ey) in mouth:
            cv2.circle(face, (ex, ey), 2, (144,238,144), -1)
    
    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(frame_interval) & 0xFF == ord('q'):
        break

# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()
